# Project: Wrangling and Analyze Data

In [1]:
# Imported packages

import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import logging
import pprint
from dotenv import load_dotenv


## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
### 1. Directly downloading the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
df_tweets_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
df_tweets_enhanced.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### 2. Using the Requests library to download the tweet image prediction (image_predictions.tsv)

In [3]:
def image_pred_download():
    # folder = "img_pred_folder"
    # if not os.path.exists(folder):
    #     os.makedirs(folder)

    image_pred_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
    image_pred_response = requests.get(image_pred_url)

    with open("image-predictions.tsv", "wb") as file:
        file.write(image_pred_response.content)

    return pd.read_csv("image-predictions.tsv", sep='\t')

df_image_pred = image_pred_download()

In [4]:
df_image_pred.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


### 3. Using the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [5]:
# Importing authentication keys

load_dotenv()
my_key = os.getenv('API_KEY')
my_secret = os.getenv('API_KEY_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
my_token = os.getenv('MY_TOKEN')


In [6]:
# Setting up basic configuration for logging so as to log status and errors during API calls/requests 

logging.basicConfig(level=logging.DEBUG, filename="log.log", filemode="w", format="%(asctime)s - %(levelname)s - %(message)s")

In [7]:
# OAuth 1.0a Authorization

auth = tweepy.OAuth1UserHandler(my_key, my_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [8]:
# OAuth 2.0 Authentication

client = tweepy.Client(bearer_token=my_token, 
                        consumer_key=my_key, 
                        consumer_secret=my_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret,
                        return_type=dict)


In [9]:
# Making a list of all the tweet ids from the given dataset

id_list = list(df_tweets_enhanced.tweet_id)


In [10]:
# Defining a function to get details tweets by id using OAuth 2.0.

def get_tweets_detailed(id_list):

    """
    I am getting these tweet fields because I plan using them
    for analysis in the future though for this current analysis,
    I will only be using the public metrics.
    """
    tweet_fields = [
        'id',
        'text',
        'created_at',
        'public_metrics',
        'source',
        'entities',
        'reply_settings',
    ]
    
    return client.get_tweets(id_list, tweet_fields=tweet_fields)


In [11]:
"""
Creating a function to get tweets in batches because of Twtter's rate limit.
Using the get_tweets method, up to 100 tweets can be gotten with a single HTTP
request. That means all 2400+ tweet data can be gotten at once (with no wait time) 
since we are making only 25 HTTP requests in total (instead of 2400+ requests)
"""

def get_tweet_in_batches(list_of_ids, batch_size=100):
    index = 0
    id_queue = []
    tweet_list = []
    while len(id_queue) == batch_size or not id_queue:
        id_queue = list_of_ids[index:(index+batch_size)]
        """
        The get_tweets that was defined above will return a dict object because that 
        is the return type that was set in the client call. It returns a dictionary 
        with a single key "data" and a list of JSON objects as the value. Each object 
        in the list corresponds to a single tweet.
        """
        logging.debug(f"Getting tweets {index} to {index+batch_size}")
        tweet_list.extend(iter(get_tweets_detailed(id_queue)['data']))
        index += batch_size

    return tweet_list


# Calling the function to get data for all tweets in the tweet list

tweets_detailed = get_tweet_in_batches(list_of_ids=id_list)

In [16]:
# Confirming the tweets' data was queried properly

pprint.pprint(tweets_detailed[0])


{'created_at': '2017-08-01T16:23:56.000Z',
 'entities': {'annotations': [{'end': 14,
                               'normalized_text': 'Phineas',
                               'probability': 0.6637,
                               'start': 8,
                               'type': 'Person'}],
              'urls': [{'display_url': 'pic.twitter.com/MgUWQ76dJU',
                        'end': 109,
                        'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
                        'media_key': '3_892420639486877696',
                        'start': 86,
                        'url': 'https://t.co/MgUWQ76dJU'}]},
 'id': '892420643555336193',
 'public_metrics': {'like_count': 33803,
                    'quote_count': 219,
                    'reply_count': 140,
                    'retweet_count': 7000},
 'reply_settings': 'everyone',
 'source': 'Twitter for iPhone',
 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole

In [17]:
# Confirming the number of tweets queried

len(tweets_detailed)

2327

In [35]:
# Saving the detailed tweet data to a text file

def save_tweets_detailed():
    with open("tweet_detailed_json.txt", "w") as file:
        file.write(json.dumps(tweets_detailed))

save_tweets_detailed()

In [38]:
# Reading the detailed tweet data from the text file

def read_tweets_detailed():
    with open("tweet_detailed_json.txt") as file:
        json_obj = json.load(file)
    return json_obj

tweets_detailed = read_tweets_detailed()

In [50]:
# Function to extract only the public metrics from the detailed tweet data

def get_tweets_data():
    tweet_data = []
    for data in tweets_detailed:
        data["public_metrics"]["tweet_id"] = data['id']
        data["public_metrics"]["created_at"] = data['created_at']
        tweet_data.append(data["public_metrics"])
    return tweet_data

def save_tweets_data():
    with open("tweet_json.txt", "w") as file:
        file.write(json.dumps(get_tweets_data()))


In [51]:
# Reading the tweets' data to a pandas dataframe

df_public_metrics = pd.read_json("tweet_json.txt")
df_public_metrics

,retweet_count,reply_count,like_count,quote_count,tweet_id,created_at
0,7000,140,33803,219,892420643555336192,2017-08-01 16:23:56+00:00
1,5298,170,29315,274,892177421306343424,2017-08-01 00:17:27+00:00
2,3477,109,22043,145,891815181378084864,2017-07-31 00:18:03+00:00
3,7222,142,36917,286,891689557279858688,2017-07-30 15:58:51+00:00
4,7753,193,35298,296,891327558926688256,2017-07-29 16:00:24+00:00
...,...,...,...,...,...,...
2322,37,9,89,0,666049248165822464,2015-11-16 00:24:50+00:00
2323,115,1,247,2,666044226329800704,2015-11-16 00:04:52+00:00
2324,36,1,100,0,666033412701032448,2015-11-15 23:21:54+00:00
2325,39,0,112,0,666029285002620928,2015-11-15 23:05:30+00:00


In [89]:
df_public_metrics.to_csv('test.csv', index=False)

In [166]:
# Function to get information for a list of tweet ids using OAuth 1.0a

def get_tweets_single(id_list):
    """
    This function makes a HTTP request for each tweet one by one
    """
    for tweet_id in id_list:
        try:
            logging.debug('get_tweets_single: fetching tweet for ID %s', tweet_id)
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print(f"{tweet_id}, {tweet.text.encode('UTF-8')}")
            print(f"{tweet_id}, {tweet.favorite_count}")
            print(f"{tweet_id}, {tweet.retweet_count}")
            return tweet
        except tweepy.TweepyException as te:
            logging.warn('get_tweets_single: failed to get tweet ID %s: %s', tweet_id, te.message)



## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



In [122]:
df_tweets_enhanced["in_reply_to_status_id"].isnull().sum()

# df_tweets_enhanced.sort_values(by=['rating_denominator']).head()
# df_tweets_enhanced.sort_values(by=['rating_denominator']).query("tweet_id == 835246439529840640").text

# df_tweets_enhanced["rating_denominator"].value_counts()
# df_tweets_enhanced.groupby('rating_numerator')["rating_numerator"].count()

df_public_metrics["like_count"].value_counts()
df_public_metrics.query("like_count == 0")

df_tweets_enhanced[df_tweets_enhanced["retweeted_status_id"].isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


### Quality issues
1. Large number of missing values in the following columns: 
	in_reply_to_status_id, 
	in_reply_to_user_id, 
	retweeted_status_id, 
	retweeted_status_user_id, 
	retweeted_status_timestamp

2. id columns (column names ending with 'id') are in numeric datatype instead of string datatype

3. Timestamp columns are string instead of datetime datatype

4. None values interpreted as string in dog stage columns

5. Some of the rating denominator values are not exactly 10

6. Some of the rating numerators values are unusually high

7. Dog prediction names are seprtated by underscores instead of spaces and they are not capitalized

8. There are 160 tweets that have 0 replys, likes and quotes but they somehow have retweets

### Tidiness issues
1. URL column (source) is still decorated with HTML "a" tag elements

2. Dog stage variable is in multiple columns instead of one
   
3. Top 3 image prediction results are in 3 seperate column sets instead of one set of columns

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization